# Presentation: 
* Retries
* Parameters - Validation
* Deployments - including Schedulling
* Task Library - GONE
* Secrets - GONE
* Async execution
* Exercise: Putting Together a Simple Data Pipeline
* Q&A

### Retries
Prefect tasks can automatically retry on failure when you enable parameters `retries` and `retry_delay_seconds` on a @task

### Deployments

### Async

In [ ]:
from prefect import task, flow
from typing import List
import httpx

@task(retries=3)
def get_stars(repo: str):
    url = f"https://api.github.com/repos/{repo}"
    count = httpx.get(url).json()["stargazers_count"]
    print(f"{repo} has {count} stars!")


@flow(name="Github stars")
def github_stars(repos: List[str]):
    for repo in repos:
        get_stars(repo)

# run the flow!
github_stars(["PrefectHQ/Prefect", "PrefectHQ/miter-design"])

In [ ]:
import prefect import task, flow
import requests
import re
from bs4 import BeautifulSoup

nikeURL = "https://www.nike.com/t/air-max-270-womens-shoes-Pgb94t/AH6789-601"

@task
def find_nike_price():
    k = requests.get(nikeURL).text
    soup = BeautifulSoup(k,'html.parser')
    price_string = soup.find('div', {"class":"product-price"}).text
    price_string = price_string.replace(' ','')
    price = int(re.search('[0-9]+',price_string).group(0))
    return price

@task
def compare_price(price, buy_price):
    if price <= buy_price:
        return True
    return False


# Notification Task sends message to Cloud once authenticated with a webhook
buy_notification, dont_buy_notification = SlackTask(
    message="BUY the nikes!",
    webhook_secret="SALES_DEMO_NOTIFICATIONS_SLACK_URL",
), SlackTask(
    message="DONT BUY the nikes!",
    webhook_secret="SALES_DEMO_NOTIFICATIONS_SLACK_URL",
)


@flow()
def nike_flow():
    price = find_nike_price()
    buy = compare_price(price.result(), 120)
    with case(buy, True):
        buy_notification()
    with case(buy, False):
        dont_buy_notification()

flow.register(project_name='AE Demos')